# Scraping Waterlevel data from Flood forecast Stations

The Central Water Commission (CWC) collects and maintains water level data from Flood forecast stations. This data is made available by the CWC here - [Link](https://ffs.india-water.gov.in/). This notebook scrapes data from flood forecast stations in Assam from this website.

In [3]:
master_url='https://ffs.india-water.gov.in/iam/api/layer-station//all/0035-CDBNG,029CDBNG,009%20%20%20%20%20-srdcbe,011-UBDDIB,015-LGDHYD,009-CDBNG,028-LGDHYD,027-LGDHYD,017-LGDHYD,049-CDBNG,031-MGD1LKN,023-mgd4ptn,006-UKDPUNE,013-lkdhyd,011-SRDCBE,022-CDBNG,014-lgdhyd,006-MGD1LKN,016-LGDHYD,013-lgdhyd,027-cdbng,0044-CDBNG,011-mgd4ptn,CW1WAR000056,012-TDSURAT,002-mgd2lkn,007-MGD1LKN,004-CDBNG,037-LKDHYD,010-MGD1LKn,039-TDSURAT,020-mgd4ptn,013-SRDCBE,003-SWRDKOCHI'
print(master_url)

https://ffs.india-water.gov.in/iam/api/layer-station//all/0035-CDBNG,029CDBNG,009%20%20%20%20%20-srdcbe,011-UBDDIB,015-LGDHYD,009-CDBNG,028-LGDHYD,027-LGDHYD,017-LGDHYD,049-CDBNG,031-MGD1LKN,023-mgd4ptn,006-UKDPUNE,013-lkdhyd,011-SRDCBE,022-CDBNG,014-lgdhyd,006-MGD1LKN,016-LGDHYD,013-lgdhyd,027-cdbng,0044-CDBNG,011-mgd4ptn,CW1WAR000056,012-TDSURAT,002-mgd2lkn,007-MGD1LKN,004-CDBNG,037-LKDHYD,010-MGD1LKn,039-TDSURAT,020-mgd4ptn,013-SRDCBE,003-SWRDKOCHI


In [3]:
station_more_details_url = 'https://ffs.india-water.gov.in/iam/api/flood-forecast-static/specification/?specification=%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22type%22,%22operator%22:%22eq%22,%22value%22:%22Level%22%7D%7D,%22or%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22type%22,%22operator%22:%22eq%22,%22value%22:%22Inflow%22%7D%7D%7D'
print(station_more_details_url)

https://ffs.india-water.gov.in/iam/api/flood-forecast-static/specification/?specification=%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22type%22,%22operator%22:%22eq%22,%22value%22:%22Level%22%7D%7D,%22or%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22type%22,%22operator%22:%22eq%22,%22value%22:%22Inflow%22%7D%7D%7D


In [4]:
print(station_address_details_url)

# When you have the station id.
station_address_details_url = 'https://ffs.india-water.gov.in/iam/api/layer-station/029-LBDJPG'

https://ffs.india-water.gov.in/iam/api/layer-station/specification/?specification=%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22floodForecastStaticStationCode.type%22,%22operator%22:%22eq%22,%22value%22:%22Level%22%7D%7D,%22or%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22floodForecastStaticStationCode.type%22,%22operator%22:%22eq%22,%22value%22:%22Inflow%22%7D%7D%7D


In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry

In [4]:
#URL containing FFS Station locations
station_location_url = 'https://ffs.india-water.gov.in/iam/api/layer-station-geo/specification/?specification=%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22layerStationStationCode.floodForecastStaticStationCode.type%22,%22operator%22:%22eq%22,%22value%22:%22Level%22%7D%7D,%22or%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22layerStationStationCode.floodForecastStaticStationCode.type%22,%22operator%22:%22eq%22,%22value%22:%22Inflow%22%7D%7D%7D'

r = requests.get(station_location_url,verify=False)
txt = r.json()
station_locations_df = pd.DataFrame(txt)
station_locations_df = station_locations_df[['name','stationCode', 'lat', 'lon']]
station_locations_df.head()

C:\Users\saikr\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffs.india-water.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,name,stationCode,lat,lon
0,Yingkiong,009NEID3,28.620000,95.030000
1,Kalisindh Dam,038-CDJAPR,24.482706,76.223006
2,Parwan pick-up Weir,040-CDJAPR,24.712894,76.555764
3,Bamni (Ballarsha),017-WGDNGP,19.814444,79.379167
4,Samdoli (Seasonal),003-UKDPUNE,16.856389,74.496667


In [5]:
station_locations_df.to_csv('station_coordinates.csv',index=False)

In [6]:
#URL containing FFS Station details

station_details_url = 'https://ffs.india-water.gov.in/iam/api/layer-station/specification/?specification=%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22floodForecastStaticStationCode.type%22,%22operator%22:%22eq%22,%22value%22:%22Level%22%7D%7D,%22or%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22floodForecastStaticStationCode.type%22,%22operator%22:%22eq%22,%22value%22:%22Inflow%22%7D%7D%7D'

r = requests.get(station_address_details_url,verify=False)
txt = r.json()
station_details_df = pd.DataFrame(txt)
station_details_df.head()


C:\Users\saikr\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffs.india-water.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,@class,stationCode,accessibility,address,airport,altitude,bank,busStand,catchmentArea,combinedWith,...,stationTypeId,streamLocalriverId,subdivisionalOfficeId,tahsilId,villageId,wellPurposeExplorationStationCode,wellPurposeId,wellSubtypeId,wellTypeId,wellUseId
0,com.eptisa.dto.SimpleLayerStationDto,009NEID3,BY road & Helicopter,,Pasighat,NaN,Left,Yingkiong,NaN,None,...,1,1247.0,1485,1121101009,NaN,None,None,None,None,None
1,com.eptisa.dto.SimpleLayerStationDto,038-CDJAPR,,,,NaN,None,,NaN,None,...,1,146.0,144,1081191004,NaN,None,None,None,None,None
2,com.eptisa.dto.SimpleLayerStationDto,040-CDJAPR,,,,NaN,None,,NaN,None,...,1,1301.0,144,1081191003,NaN,None,None,None,None,None
3,com.eptisa.dto.SimpleLayerStationDto,017-WGDNGP,,,,171.28,Left,,46020.0,None,...,1,243.0,119,1271131003,NaN,None,None,None,None,None
4,com.eptisa.dto.SimpleLayerStationDto,003-UKDPUNE,CONNECTED BY ROAD,"OFFICE OF JUNIOR ENGINEER, CENTRAL WATER COMMI...",KOLHAPUR - 47KM,538.00,Left,SAMDOLI,1948.0,None,...,1,290.0,91,1271351005,NaN,None,None,None,None,None


In [7]:
station_details_df.to_csv('station_details.csv',index=False)

In [8]:
assam_polygon = gpd.read_file('Assam.geojson')

In [9]:
stations = pd.read_csv('station_coordinates.csv')

# creating a geometry column 
geometry = [Point(xy) for xy in zip(stations['lon'], stations['lat'])]# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}# Creating a Geographic data frame 
stations_gdf = gpd.GeoDataFrame(stations, crs=crs, geometry=geometry)

#Spatial join with assam state
gpd.sjoin(stations_gdf.to_crs('EPSG:4326'),assam_polygon.to_crs('EPSG:4326'),how="inner", op="within").to_file('assam_stations.geojson',driver='GeoJSON')

C:\Users\saikr\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\saikr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [11]:
assam_stations_gdf = gpd.read_file('assam_stations.geojson')
assam_stations_gdf.head()

,name,stationCode,lat,lon,index_right,ST_NM,geometry
0,Annapurnaghat,010-MBDGHY,24.832778,92.791667,0,Assam,POINT (92.79167 24.83278)
1,Numaligarh,024-UBDDIB,26.631181,93.730419,0,Assam,POINT (93.73042 26.63118)
2,Tezpur,031-UBDDIB,26.615833,92.796944,0,Assam,POINT (92.79694 26.61583)
3,Dharamtul,034-UBDDIB,26.168333,92.358611,0,Assam,POINT (92.35861 26.16833)
4,Goalpara,002-MBDGHY,26.197222,90.638889,0,Assam,POINT (90.63889 26.19722)


In [34]:
startDate = '2016-01-01'
endDate = '2022-07-30'

master_df = pd.read_csv('Waterlevel_assam_stations.csv')

for stationCode in list(assam_stations_df.stationCode):
    print(stationCode)
    dynamic_url = 'https://ffs.india-water.gov.in/iam/api/new-entry-data/specification/sorted?sort-criteria=%7B%22sortOrderDtos%22:%5B%7B%22sortDirection%22:%22ASC%22,%22field%22:%22id.dataTime%22%7D%5D%7D&specification=%7B%22where%22:%7B%22where%22:%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.stationCode%22,%22operator%22:%22eq%22,%22value%22:%22'+stationCode+'%22%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.datatypeCode%22,%22operator%22:%22eq%22,%22value%22:%22HHS%22%7D%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22dataValue%22,%22operator%22:%22null%22,%22value%22:%22false%22%7D%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.dataTime%22,%22operator%22:%22btn%22,%22value%22:%22'+startDate+'T16:49:44.574,'+endDate+'T16:49:44.574%22%7D%7D%7D'
    r = requests.get(dynamic_url,verify=False)
    txt = r.json()
    df = pd.DataFrame(txt)
    df['Date'] = df.id.apply(lambda x: x['dataTime'].split('T')[0])
    df['Time'] = df.id.apply(lambda x: x['dataTime'].split('T')[1])
    df = df[['stationCode','Date','Time','dataValue','datatypeCode']]
    master_df = pd.concat([master_df,df],ignore_index=True)
    master_df.to_csv('Waterlevel_assam_stations.csv',index=False)


master_df


01-11-01-002


C:\Users\saikr\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffs.india-water.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


01-11-01-008


C:\Users\saikr\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffs.india-water.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BV000FS


...

01-11-03-001


C:\Users\saikr\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffs.india-water.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


012-MBDGHY


C:\Users\saikr\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffs.india-water.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,stationCode,Date,Time,dataValue,datatypeCode
0,010-MBDGHY,2016-04-07,00:00:00,17.47,HHS
1,010-MBDGHY,2016-04-07,01:00:00,17.44,HHS
2,010-MBDGHY,2016-04-07,02:00:00,17.40,HHS
3,010-MBDGHY,2016-04-07,03:00:00,17.36,HHS
4,010-MBDGHY,2016-04-07,04:00:00,17.32,HHS
...,...,...,...,...,...
2124068,012-MBDGHY,2022-07-26,17:00:00,25.04,HHS
2124069,012-MBDGHY,2022-07-26,18:00:00,25.03,HHS
2124070,012-MBDGHY,2022-07-26,19:00:00,25.03,HHS
2124071,012-MBDGHY,2022-07-26,20:00:00,25.03,HHS


## References:
1. https://pib.gov.in/newsite/PrintRelease.aspx?relid=181066
2. http://jalshakti-dowr.gov.in/schemes-projects-programmes/schemes/flood-forecasting
3. https://indiawris.gov.in/wiki/doku.php?id=cwc_national_flood_forecasting_network
4. http://iced.cag.gov.in/wp-content/uploads/2016-17/NTP%2002/VD%20Roy.pdf
5. http://cwc.gov.in/sites/default/files/final-appraisal-report-2020-publication-no..pdf

There are three divisions in Assam:
1. Upper Brahmaputra Division, Dibrugarh (UBDDIB): As per 2020, 19 stations
2. Middle Brahmaputra Divison, Guwahati (MBDGHY): As per 2020, 15 stations
3. Lower Brahmaputra Division, Jalpaiguri (LBDJPG): As per 2020, 8 stations

In [12]:
master_df = pd.read_csv('Waterlevel_assam_stations.csv')
master_df

,stationCode,Date,Time,dataValue,datatypeCode
0,010-MBDGHY,2016-04-07,00:00:00,17.47,HHS
1,010-MBDGHY,2016-04-07,01:00:00,17.44,HHS
2,010-MBDGHY,2016-04-07,02:00:00,17.40,HHS
3,010-MBDGHY,2016-04-07,03:00:00,17.36,HHS
4,010-MBDGHY,2016-04-07,04:00:00,17.32,HHS
...,...,...,...,...,...
2124068,012-MBDGHY,2022-07-26,17:00:00,25.04,HHS
2124069,012-MBDGHY,2022-07-26,18:00:00,25.03,HHS
2124070,012-MBDGHY,2022-07-26,19:00:00,25.03,HHS
2124071,012-MBDGHY,2022-07-26,20:00:00,25.03,HHS
